# G2 Correlation Analysis Workflow

This notebook demonstrates the complete G2 correlation analysis workflow:
from generating synthetic data, to fitting with NLSQ, visualization,
and extracting physical parameters (diffusion coefficient).

## Prerequisites

- xpcsviewer installed (`uv sync` or `pip install -e .`)

> **Note**: This notebook uses synthetic data so it runs without external HDF5 files.
> In practice, you would load data via `XpcsFile` as shown in [01_loading_data.ipynb](01_loading_data.ipynb).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

%matplotlib inline
plt.rcParams['figure.dpi'] = 100
plt.rcParams['figure.figsize'] = (10, 6)

## 1. Generate Synthetic Multi-Q G2 Data

In [ ]:
np.random.seed(42)

# Physical parameters: Brownian diffusion
D_true = 5e3              # True diffusion coefficient (nm^2/s)
baseline_true = 1.0
contrast_true = 0.25
n_q = 12
n_t = 60

q_values = np.linspace(0.008, 0.04, n_q)
labels = [f"q={q:.4f} nm\u207b\u00b9" for q in q_values]
t_el = np.logspace(-3, 2, n_t)

# tau(q) = 1 / (D * q^2) -- diffusive dynamics
tau_true = 1.0 / (D_true * q_values**2)

g2 = np.zeros((n_t, n_q))
g2_err = np.zeros((n_t, n_q))

for i in range(n_q):
    g2_clean = baseline_true + contrast_true * np.exp(-2 * t_el / tau_true[i])
    noise = 0.004 + 0.002 * np.random.rand()
    g2[:, i] = g2_clean + np.random.normal(0, noise, size=n_t)
    g2_err[:, i] = noise

print(f"G2 data: {n_q} Q bins, {n_t} delay points")
print(f"Delay time range: [{t_el.min():.2e}, {t_el.max():.2e}] s")
print(f"Q range: [{q_values.min():.4f}, {q_values.max():.4f}] nm\u207b\u00b9")
print(f"True D = {D_true:.0f} nm\u00b2/s")

## 2. Overview: All Q Bins

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

colors = plt.cm.viridis(np.linspace(0, 1, n_q))
for i in range(n_q):
    ax.semilogx(t_el, g2[:, i], "o-", color=colors[i], markersize=2, linewidth=0.5)

ax.set_xlabel("Delay time (s)")
ax.set_ylabel(r"$g_2(\tau)$")
ax.set_title(f"G2 Overview ({n_q} Q bins)")

sm = plt.cm.ScalarMappable(cmap="viridis", norm=plt.Normalize(q_values.min(), q_values.max()))
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label(r"Q (nm$^{-1}$)")
plt.tight_layout()
plt.show()

## 3. Single Q-Bin Analysis

In [ ]:
q_idx = 5

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].errorbar(t_el, g2[:, q_idx], yerr=g2_err[:, q_idx],
                 fmt="o", markersize=4, capsize=2)
axes[0].set_xscale("log")
axes[0].set_xlabel("Delay time (s)")
axes[0].set_ylabel(r"$g_2(\tau)$")
axes[0].set_title(f"G2: {labels[q_idx]}")

axes[1].hist(g2_err[:, q_idx], bins=30, edgecolor="black", alpha=0.7)
axes[1].set_xlabel("G2 Error")
axes[1].set_ylabel("Count")
axes[1].set_title(f"Error Distribution: {labels[q_idx]}")

plt.tight_layout()
plt.show()

## 4. G2 Fitting with NLSQ

Use the `nlsq_fit()` function for per-Q single-exponential fitting.

Model: $g_2(\tau) = \text{baseline} + \text{contrast} \cdot \exp(-2\tau / \tau_r)$

In [ ]:
from xpcsviewer.fitting import nlsq_fit
from xpcsviewer.fitting.models import single_exp_func

# Fit each Q bin independently
fit_params = []  # (tau, baseline, contrast) per Q
fit_errors = []  # uncertainties per Q
fit_lines = []   # fitted curves

t_fine = np.logspace(np.log10(t_el.min()), np.log10(t_el.max()), 200)

for i in range(n_q):
    result = nlsq_fit(
        model_fn=single_exp_func,
        x=t_el,
        y=g2[:, i],
        yerr=g2_err[:, i],
        p0={"tau": 1.0, "baseline": 1.0, "contrast": 0.2},
        bounds={
            "tau": (1e-6, 1e4),
            "baseline": (0.5, 2.0),
            "contrast": (0.0, 1.0),
        },
    )
    fit_params.append(result.params)
    fit_errors.append({
        name: result.get_param_uncertainty(name) for name in result.params
    })
    fit_lines.append(single_exp_func(
        t_fine, result.params["tau"], result.params["baseline"], result.params["contrast"]
    ))

# Extract arrays
tau_fitted = np.array([float(p["tau"]) for p in fit_params])
baseline_fitted = np.array([float(p["baseline"]) for p in fit_params])
contrast_fitted = np.array([float(p["contrast"]) for p in fit_params])
tau_err = np.array([float(e["tau"]) for e in fit_errors])

print("Fitted parameters per Q bin:")
print(f"{'Q':>10} {'tau':>12} {'baseline':>10} {'contrast':>10}")
print("-" * 45)
for i in range(n_q):
    print(f"{q_values[i]:10.4f} {tau_fitted[i]:12.4e} {baseline_fitted[i]:10.4f} {contrast_fitted[i]:10.4f}")

In [ ]:
n_plot = min(6, n_q)
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i in range(n_plot):
    ax = axes[i]
    ax.errorbar(t_el, g2[:, i], yerr=g2_err[:, i],
                fmt="o", markersize=3, capsize=1, label="Data")
    ax.plot(t_fine, np.asarray(fit_lines[i]), "r-", linewidth=2, label="Fit")
    ax.set_xscale("log")
    ax.set_xlabel("Delay time (s)")
    ax.set_ylabel(r"$g_2(\tau)$")
    ax.set_title(f"{labels[i]}", fontsize=10)
    ax.legend(fontsize=8)

plt.suptitle("G2 Single-Exponential Fits", fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

## 5. Tau vs Q Analysis

For diffusive dynamics: $1/\tau = D \cdot q^2$

In [ ]:
valid = (tau_fitted > 0) & np.isfinite(tau_fitted)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# tau vs q
axes[0].semilogy(q_values[valid], tau_fitted[valid], "bo-")
axes[0].set_xlabel(r"$q$ (nm$^{-1}$)")
axes[0].set_ylabel(r"$\tau$ (s)")
axes[0].set_title(r"$\tau$ vs $q$")

# 1/tau vs q^2
axes[1].plot(q_values[valid]**2, 1.0 / tau_fitted[valid], "ro-")
axes[1].set_xlabel(r"$q^2$ (nm$^{-2}$)")
axes[1].set_ylabel(r"$1/\tau$ (s$^{-1}$)")
axes[1].set_title(r"$1/\tau$ vs $q^2$")

if valid.sum() > 2:
    coeffs = np.polyfit(q_values[valid]**2, 1.0 / tau_fitted[valid], 1)
    D_fit = coeffs[0]
    q2_fine = np.linspace(0, q_values[valid].max()**2 * 1.1, 100)
    axes[1].plot(q2_fine, np.polyval(coeffs, q2_fine), "k--",
                 label=f"D = {D_fit:.0f} nm$^2$/s")
    axes[1].legend()
    print(f"Fitted diffusion coefficient: D = {D_fit:.0f} nm\u00b2/s")
    print(f"True diffusion coefficient:   D = {D_true:.0f} nm\u00b2/s")

# Contrast vs q
axes[2].plot(q_values[valid], contrast_fitted[valid], "gs-")
axes[2].set_xlabel(r"$q$ (nm$^{-1}$)")
axes[2].set_ylabel("Contrast")
axes[2].set_title(r"Contrast vs $q$")

plt.tight_layout()
plt.show()

## 6. SAXS 1D Profile

In [ ]:
# Synthetic SAXS data
q_saxs = np.logspace(-2, -0.5, 200)
Iq_saxs = 1e4 * q_saxs**(-2.5) * (1 + np.random.normal(0, 0.02, size=q_saxs.shape))

fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(q_saxs, Iq_saxs, "-", linewidth=1)
ax.set_xlabel(r"Q (nm$^{-1}$)")
ax.set_ylabel("I(Q) (a.u.)")
ax.set_title("SAXS 1D Profile")
plt.tight_layout()
plt.show()

## 7. Heatmap Visualization

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

im = ax.pcolormesh(
    np.arange(g2.shape[1]),
    t_el,
    g2,
    shading="auto",
    cmap="viridis",
)
ax.set_yscale("log")
ax.set_xlabel("Q bin index")
ax.set_ylabel("Delay time (s)")
ax.set_title("G2 Heatmap")
plt.colorbar(im, ax=ax, label=r"$g_2(\tau)$")
plt.tight_layout()
plt.show()

## Summary

This notebook covered:

1. Generating synthetic G2 data with diffusive dynamics
2. Visualizing G2 curves across Q bins (waterfall, heatmap)
3. Single Q-bin analysis and error distributions
4. Per-Q single-exponential fitting with `nlsq_fit()`
5. Extracting tau(q) for diffusion coefficient estimation
6. SAXS 1D profile visualization

Next: [03_fitting_workflow.ipynb](03_fitting_workflow.ipynb) for advanced fitting with Bayesian methods.